In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time

data_base_dir = '../../datasets/Movielens/'
data_dir2 = data_base_dir + 'Movielens Latest/ml-latest/'
data_dir = data_base_dir + 'ml-20m/'

genome_scores = data_dir2 + 'genome-scores.csv'
genome_tags = data_dir + 'genome-tags.csv'
movies = data_dir + 'movies.csv'
ratings = data_dir + 'ratings.csv'
tags = data_dir + 'tags.csv'
genres = data_dir + 'u.genre'


user_full_genome_vector_df = pd.read_pickle(data_dir + 'output/user_full_genome_terms_df_gzip', compression='gzip')
user_lemmatized_genome_vector_df = pd.read_pickle(data_dir + 'output/user_lemmatized_genome_terms_df_gzip', compression='gzip')

movie_full_genome_scores_df = pd.read_csv(genome_scores).pivot(index='movieId', columns='tagId', values='relevance')
movie_lemmatized_genome_scores_df = pd.read_pickle(data_dir + 'output/movies_lemmatized_genome_vector_df_bz2', compression='bz2')

In [26]:
from sklearn.metrics import pairwise_distances
main_model_lemmatized = [8125, 25930, 1927, 26122, 559, 5151, 59022, 101281, 50189, 5781, 6550, 85367, 7239, 6036, 72407, 94677, 44193, 74752, 104441, 80363]
genre_binary = [100553,    913,   1147,   1189  , 2859, 120478,   2066   ,7156,   1361  , 3077,8228  , 3677   , 246 , 32853 , 58425 ,  6896,  85774  , 7566    ,602    ,162]
genre_integer=[100553,    913,   1147   , 1189   , 2859 ,120478  , 2066  , 7156,   1361,   3077, 8228 ,  3677,    246  , 32853  , 58425  , 6896  ,85774   ,7566,602, 162]
genome_lemmatized=[  1315 ,115414,  27114   ,3804 ,110809  ,74075,   6036,   3216, 104736,  73106, 59429   ,5687 , 77414   ,4872  , 3691   ,2894  , 6037,   2234  ,  559, 111387]
genome_full = [  1315,  77414,   5687,  27114,  43930,  88108,   8911, 104736,  95199,  78264, 26791, 115414,  74154 , 65193, 110771,  31433,  63393 ,109189   ,8869,54734]

user_id = 1

user_scores_df = movie_lemmatized_genome_scores_df.loc[main_model_lemmatized, :]
user_vector_df = user_lemmatized_genome_vector_df.loc[user_id, :].values.reshape(1, -1)

main_model_lemmatized_df = pd.DataFrame(index=main_model_lemmatized)
main_model_lemmatized_df['similarity'] = pairwise_distances(user_vector_df, user_scores_df.values, metric='cosine').reshape(-1, 1)
main_model_lemmatized_df['diversity'] = 1 - main_model_lemmatized_df['similarity']
main_model_lemmatized_df
# pairwise_distances(user_vector_df, user_scores_df.values, metric='cosine')
# pd.DataFrame(index=main_model_lemmatized)

,similarity,diversity
8125,0.202940,0.797060
25930,0.204947,0.795053
1927,0.212970,0.787030
26122,0.217605,0.782395
559,0.333065,0.666935
5151,0.298325,0.701675
59022,0.291803,0.708197
101281,0.328249,0.671751
50189,0.329570,0.670430
5781,0.297519,0.702481


In [47]:
# average similarity to user
print("average similarity: ", main_model_lemmatized_df['similarity'].mean())
print("average diversity", main_model_lemmatized_df['diversity'].mean())

# diversity of list
all_movies = main_model_lemmatized_df.index.values
movie_genomes_df = movie_lemmatized_genome_scores_df.loc[all_movies, :]

m_distances_df = pd.DataFrame(1 - pairwise_distances(movie_genomes_df.values, movie_genomes_df.values, metric='cosine'), index=all_movies, columns=all_movies)
m_distances_df

average similarity:  0.28783753620917113
average diversity 0.7121624637908288


,8125,25930,1927,26122,559,5151,59022,101281,50189,5781,6550,85367,7239,6036,72407,94677,44193,74752,104441,80363
8125,1.000000,0.914699,0.778498,0.855931,0.531202,0.489858,0.480813,0.510782,0.475585,0.493057,0.457234,0.492586,0.502578,0.474824,0.443494,0.490532,0.471785,0.484804,0.543943,0.450181
25930,0.914699,1.000000,0.835283,0.851746,0.538823,0.503985,0.493479,0.518308,0.490295,0.503819,0.482124,0.475183,0.508486,0.486375,0.422173,0.511122,0.464607,0.499075,0.580670,0.435777
1927,0.778498,0.835283,1.000000,0.790922,0.500389,0.499262,0.500429,0.495426,0.493473,0.509422,0.461780,0.472342,0.513635,0.487286,0.486959,0.503496,0.483235,0.492160,0.517818,0.446362
26122,0.855931,0.851746,0.790922,1.000000,0.622811,0.541394,0.521452,0.514121,0.533021,0.567767,0.477671,0.479708,0.489905,0.527962,0.419408,0.532576,0.467667,0.553115,0.579825,0.442220
559,0.531202,0.538823,0.500389,0.622811,1.000000,0.787922,0.704093,0.611525,0.833406,0.849017,0.532390,0.575734,0.488205,0.849203,0.516642,0.614756,0.565861,0.730266,0.647981,0.528691
5151,0.489858,0.503985,0.499262,0.541394,0.787922,1.000000,0.846220,0.836200,0.922441,0.877586,0.758261,0.857062,0.533218,0.850184,0.682315,0.813153,0.841132,0.835293,0.641982,0.617971
59022,0.480813,0.493479,0.500429,0.521452,0.704093,0.846220,1.000000,0.775254,0.860303,0.764408,0.793824,0.769867,0.527956,0.734871,0.610615,0.887905,0.724775,0.816395,0.584914,0.622118
101281,0.510782,0.518308,0.495426,0.514121,0.611525,0.836200,0.775254,1.000000,0.718032,0.654225,0.793809,0.909501,0.496330,0.648761,0.592314,0.816602,0.772410,0.697263,0.537433,0.531063
50189,0.475585,0.490295,0.493473,0.533021,0.833406,0.922441,0.860303,0.718032,1.000000,0.916212,0.699528,0.710823,0.515835,0.918221,0.632981,0.768065,0.768164,0.837246,0.619920,0.595327
5781,0.493057,0.503819,0.509422,0.567767,0.849017,0.877586,0.764408,0.654225,0.916212,1.000000,0.651243,0.653852,0.525553,0.930957,0.672872,0.687357,0.766882,0.833696,0.646028,0.585484


In [48]:
# diversity of a list
m_distances_df.sum(axis=1).sum() * (1/(20 * 19))

0.6774592991028322